# 03 - Encoding y Transformaciones Avanzadas ENCSPA 2019
## Pipeline de Preprocesamiento para Machine Learning

**Objetivo:** Implementar transformaciones avanzadas y técnicas de balanceo para preparar los datos para modelos de Machine Learning.

**Tareas del Commit 3:**
- ColumnTransformer con OneHotEncoder y StandardScaler
- SMOTE para balancear clases desbalanceadas (8% prevalencia marihuana)
- Pipeline completo de preprocesamiento
- Validación del pipeline con métricas

**Variables objetivo identificadas:**
- Marihuana (G_11_F): Variable principal - 8.0% prevalencia

**Variables predictoras (15 total):**
- 5 categóricas: Entorno social y actitudes (G_01, G_02, G_03, G_04, G_05)
- 10 numéricas: Cantidades, accesibilidad y exposición (G_01_A, G_02_A, G_06_A-D, G_07, G_08_A-B)

## Pasos del encoding y transformaciones (Tercer Commit)

**Fase 1 - Carga de datos preprocesados:**
1. Cargar datos limpios del commit anterior
2. Verificar integridad de variables seleccionadas
3. Confirmar división train/test estratificada

**Fase 2 - Transformaciones avanzadas:**
4. ColumnTransformer con OneHotEncoder para categóricas
5. StandardScaler para variables numéricas
6. Pipeline completo de preprocesamiento

**Fase 3 - Balanceo de clases:**
7. SMOTE para balancear datos de entrenamiento
8. Análisis del impacto del balanceo
9. Validación final del pipeline

*Nota: Modelos de Machine Learning se implementarán en el siguiente commit*

In [1]:
# Importación de librerías (siguiendo estructura del profesor)
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Preprocesamiento y transformaciones
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler, LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer

# Balanceo de clases
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline as ImbPipeline

# Métricas y validación
from sklearn.metrics import classification_report, confusion_matrix
from collections import Counter

# Configuración
import warnings
warnings.filterwarnings('ignore')

# Configurar matplotlib para español
plt.rcParams['font.size'] = 12
sns.set_style("whitegrid")

print("Librerías importadas correctamente")
print("Notebook 03: Encoding y Transformaciones Avanzadas")

Librerías importadas correctamente
Notebook 03: Encoding y Transformaciones Avanzadas


## 1. Carga y verificación de datos preprocesados

In [2]:
# Cargar dataset original (mismo proceso que commits anteriores)
df_original = pd.read_csv('../data/g_capitulos.csv')

print(f"Dataset ENCSPA 2019 cargado")
print(f"Dimensiones: {df_original.shape}")
print(f"Observaciones: {df_original.shape[0]:,}")
print(f"Variables: {df_original.shape[1]}")

# Verificar que tenemos los datos correctos
print(f"\nVerificación de integridad:")
print(f"   Datos originales cargados correctamente")
print(f"   Listo para aplicar filtros y transformaciones")

Dataset ENCSPA 2019 cargado
Dimensiones: (49756, 98)
Observaciones: 49,756
Variables: 98

Verificación de integridad:
   Datos originales cargados correctamente
   Listo para aplicar filtros y transformaciones


## 2. Definición de variables y sistema de nombres legibles

In [3]:
# Variables seleccionadas basadas en commits anteriores
# IMPORTANTE: Estas variables deben coincidir exactamente con notebooks 01 y 02

# Variable objetivo principal
target_variable = 'G_11_F'  # Consumo de marihuana (8% prevalencia)

# Variables predictoras categóricas (entorno social + actitudes)
categorical_features = [
    # Entorno Social
    'G_01',  # Familiares consumen sustancias
    'G_02',  # Amigos consumen sustancias  
    
    # Actitudes y Curiosidad
    'G_03',  # Curiosidad por probar
    'G_04',  # Disposición a consumir
    'G_05'   # Tuvo oportunidad de probar
]

# Variables predictoras numéricas (accesibilidad + exposición)
numerical_features = [
    # Cantidad de familiares/amigos
    'G_01_A',  # Cantidad de familiares que consumen
    'G_02_A',  # Cantidad de amigos que consumen
    
    # Accesibilidad (facilidad de acceso)
    'G_06_A',  # Facilidad acceso marihuana
    'G_06_B',  # Facilidad acceso cocaína
    'G_06_C',  # Facilidad acceso basuco
    'G_06_D',  # Facilidad acceso éxtasis
    
    # Exposición (ofertas recibidas)
    'G_07',    # Ofertas recibidas en último año
    'G_08_A',  # Ofertas de marihuana
    'G_08_B'   # Ofertas de cocaína
]

# Diccionario de nombres legibles (consistente con notebooks anteriores)
variable_names = {
    # Variable objetivo
    'G_11_F': 'Consumo de Marihuana',
    
    # Variables categóricas - Entorno Social
    'G_01': 'Familiares Consumen Sustancias',
    'G_02': 'Amigos Consumen Sustancias',
    
    # Variables categóricas - Actitudes
    'G_03': 'Curiosidad por Probar',
    'G_04': 'Disposición a Consumir',
    'G_05': 'Tuvo Oportunidad de Probar',
    
    # Variables numéricas - Cantidades
    'G_01_A': 'Cantidad de Familiares que Consumen',
    'G_02_A': 'Cantidad de Amigos que Consumen',
    
    # Variables numéricas - Accesibilidad
    'G_06_A': 'Acceso Fácil a Marihuana',
    'G_06_B': 'Acceso Fácil a Cocaína',
    'G_06_C': 'Acceso Fácil a Basuco',
    'G_06_D': 'Acceso Fácil a Éxtasis',
    
    # Variables numéricas - Exposición
    'G_07': 'Ofertas Recibidas (Último Año)',
    'G_08_A': 'Ofertas de Marihuana',
    'G_08_B': 'Ofertas de Cocaína'
}

# Función para obtener nombre legible (consistente con notebooks anteriores)
def get_readable_name(var_code):
    """Convierte código de variable a nombre legible"""
    return variable_names.get(var_code, var_code)

# Todas las variables para el análisis
all_features = categorical_features + numerical_features + [target_variable]

print(f"Variable objetivo: {get_readable_name(target_variable)}")
print(f"Variables categóricas: {len(categorical_features)}")
print(f"Variables numéricas: {len(numerical_features)}")
print(f"Total variables seleccionadas: {len(all_features)}")
print(f"Total variables predictoras: {len(categorical_features + numerical_features)}")

print(f"\nVariables categóricas (Entorno + Actitudes):")
for i, var in enumerate(categorical_features, 1):
    print(f"   {i}. {get_readable_name(var)} ({var})")

print(f"\nVariables numéricas (Cantidades + Acceso + Exposición):")
for i, var in enumerate(numerical_features, 1):
    print(f"   {i}. {get_readable_name(var)} ({var})")

print(f"\nSistema de nombres legibles implementado correctamente")
print(f"Consistente con notebooks 01 y 02")

Variable objetivo: Consumo de Marihuana
Variables categóricas: 5
Variables numéricas: 9
Total variables seleccionadas: 15
Total variables predictoras: 14

Variables categóricas (Entorno + Actitudes):
   1. Familiares Consumen Sustancias (G_01)
   2. Amigos Consumen Sustancias (G_02)
   3. Curiosidad por Probar (G_03)
   4. Disposición a Consumir (G_04)
   5. Tuvo Oportunidad de Probar (G_05)

Variables numéricas (Cantidades + Acceso + Exposición):
   1. Cantidad de Familiares que Consumen (G_01_A)
   2. Cantidad de Amigos que Consumen (G_02_A)
   3. Acceso Fácil a Marihuana (G_06_A)
   4. Acceso Fácil a Cocaína (G_06_B)
   5. Acceso Fácil a Basuco (G_06_C)
   6. Acceso Fácil a Éxtasis (G_06_D)
   7. Ofertas Recibidas (Último Año) (G_07)
   8. Ofertas de Marihuana (G_08_A)
   9. Ofertas de Cocaína (G_08_B)

Sistema de nombres legibles implementado correctamente
Consistente con notebooks 01 y 02


## 3. Filtrado y limpieza de datos

In [4]:
# Aplicar filtrado
print("Aplicando filtros de limpieza")
print("========================================")

# 1. Seleccionar solo las variables necesarias
df_subset = df_original[all_features].copy()
print(f"Subset creado: {df_subset.shape}")

# 2. Filtrar casos válidos para variable objetivo
# Mantener solo respuestas válidas (1=Sí, 2=No) para marihuana
valid_target_mask = df_subset[target_variable].isin([1, 2])
df_clean = df_subset[valid_target_mask].copy()
print(f"Después de filtrar variable objetivo: {df_clean.shape}")

# 3. Convertir variable objetivo a binaria (1=Consume, 0=No consume)
df_clean[target_variable] = (df_clean[target_variable] == 1).astype(int)
prevalencia = df_clean[target_variable].mean() * 100
print(f"Variable objetivo convertida a binaria")
print(f"Prevalencia de consumo: {prevalencia:.1f}%")

# 4. Limpiar variables categóricas (convertir 9='No sabe' a NaN)
for var in categorical_features:
    if var in df_clean.columns:
        # Convertir 9 (No sabe) a NaN
        df_clean[var] = df_clean[var].replace(9, np.nan)
        # Convertir a binario: 1=Sí, 2=No -> 1=Sí, 0=No
        df_clean[var] = (df_clean[var] == 1).astype(float)

# 5. Limpiar variables numéricas (convertir 99/999 a NaN)
for var in numerical_features:
    if var in df_clean.columns:
        # Convertir valores de 'No sabe' a NaN
        df_clean[var] = df_clean[var].replace([99, 999], np.nan)

# 6. Eliminar filas con demasiados valores faltantes
# Mantener filas que tengan al menos 80% de datos válidos
threshold = len(categorical_features + numerical_features) * 0.8
df_clean = df_clean.dropna(thresh=threshold)

print(f"\nDatos después de limpieza: {df_clean.shape}")
print(f"Casos eliminados: {df_subset.shape[0] - df_clean.shape[0]:,}")
print(f"Casos válidos finales: {df_clean.shape[0]:,}")

# 7. Análisis de valores faltantes
print(f"\nAnálisis de valores faltantes:")
missing_summary = df_clean.isnull().sum()
missing_pct = (missing_summary / len(df_clean)) * 100

for var in all_features:
    if var in df_clean.columns and missing_summary[var] > 0:
        print(f"   {get_readable_name(var)}: {missing_summary[var]} ({missing_pct[var]:.1f}%)")

if missing_summary.sum() == 0:
    print("   No hay valores faltantes en el dataset limpio")

print(f"\nDatos listos para transformaciones avanzadas")

Aplicando filtros de limpieza
Subset creado: (49756, 15)
Después de filtrar variable objetivo: (49756, 15)
Variable objetivo convertida a binaria
Prevalencia de consumo: 8.0%

Datos después de limpieza: (23427, 15)
Casos eliminados: 26,329
Casos válidos finales: 23,427

Análisis de valores faltantes:
   Cantidad de Familiares que Consumen: 13669 (58.3%)
   Cantidad de Amigos que Consumen: 9862 (42.1%)
   Ofertas de Marihuana: 9173 (39.2%)
   Ofertas de Cocaína: 9173 (39.2%)

Datos listos para transformaciones avanzadas


## 4. División train/test estratificada

In [5]:
# Preparar features y target
print("División train/test estratificada")
print("========================================")

# Separar features y target
features_para_modelo = categorical_features + numerical_features
X = df_clean[features_para_modelo].copy()
y = df_clean[target_variable].copy()

print(f"Features preparadas: {X.shape}")
print(f"Target preparado: {y.shape}")
print(f"Prevalencia general: {y.mean()*100:.1f}%")

# División train/test estratificada (80/20)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, 
    test_size=0.2, 
    random_state=42, 
    stratify=y  # Mantener proporción de clases
)

print(f"\nDivisión completada:")
print(f"   Entrenamiento: {X_train.shape[0]:,} muestras ({X_train.shape[0]/len(X)*100:.0f}%)")
print(f"   Prueba: {X_test.shape[0]:,} muestras ({X_test.shape[0]/len(X)*100:.0f}%)")
print(f"   Prevalencia train: {y_train.mean()*100:.1f}%")
print(f"   Prevalencia test: {y_test.mean()*100:.1f}%")

# Verificar distribución de clases
train_counts = Counter(y_train)
test_counts = Counter(y_test)

print(f"\nDistribución de clases:")
print(f"   Train - No consume: {train_counts[0]:,}, Consume: {train_counts[1]:,}")
print(f"   Test  - No consume: {test_counts[0]:,}, Consume: {test_counts[1]:,}")

print(f"\nDatos listos para ColumnTransformer y SMOTE")

División train/test estratificada
Features preparadas: (23427, 14)
Target preparado: (23427,)
Prevalencia general: 16.1%

División completada:
   Entrenamiento: 18,741 muestras (80%)
   Prueba: 4,686 muestras (20%)
   Prevalencia train: 16.1%
   Prevalencia test: 16.1%

Distribución de clases:
   Train - No consume: 15,721, Consume: 3,020
   Test  - No consume: 3,931, Consume: 755

Datos listos para ColumnTransformer y SMOTE


## 5. ColumnTransformer: OneHotEncoder y StandardScaler

In [6]:
# Implementar ColumnTransformer con OneHotEncoder y StandardScaler
print("Implementando ColumnTransformer")
print("========================================")

# Definir transformadores para cada tipo de variable
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),  # Imputar valores faltantes
    ('onehot', OneHotEncoder(drop='first', sparse_output=False))  # One-hot encoding
])

numerical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),  # Imputar valores faltantes
    ('scaler', StandardScaler())  # Normalización
])

# Crear ColumnTransformer
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', categorical_transformer, categorical_features),
        ('num', numerical_transformer, numerical_features)
    ],
    remainder='drop'  # Eliminar columnas no especificadas
)

print(f"ColumnTransformer configurado:")
print(f"   Variables categóricas: {len(categorical_features)} -> OneHotEncoder")
print(f"   Variables numéricas: {len(numerical_features)} -> StandardScaler")

# Mostrar detalles de las transformaciones
print(f"\nTransformaciones categóricas:")
for i, var in enumerate(categorical_features, 1):
    print(f"   {i}. {get_readable_name(var)} -> One-Hot Encoding")

print(f"\nTransformaciones numéricas:")
for i, var in enumerate(numerical_features, 1):
    print(f"   {i}. {get_readable_name(var)} -> Standardización")

print(f"\nColumnTransformer listo para entrenamiento")

Implementando ColumnTransformer
ColumnTransformer configurado:
   Variables categóricas: 5 -> OneHotEncoder
   Variables numéricas: 9 -> StandardScaler

Transformaciones categóricas:
   1. Familiares Consumen Sustancias -> One-Hot Encoding
   2. Amigos Consumen Sustancias -> One-Hot Encoding
   3. Curiosidad por Probar -> One-Hot Encoding
   4. Disposición a Consumir -> One-Hot Encoding
   5. Tuvo Oportunidad de Probar -> One-Hot Encoding

Transformaciones numéricas:
   1. Cantidad de Familiares que Consumen -> Standardización
   2. Cantidad de Amigos que Consumen -> Standardización
   3. Acceso Fácil a Marihuana -> Standardización
   4. Acceso Fácil a Cocaína -> Standardización
   5. Acceso Fácil a Basuco -> Standardización
   6. Acceso Fácil a Éxtasis -> Standardización
   7. Ofertas Recibidas (Último Año) -> Standardización
   8. Ofertas de Marihuana -> Standardización
   9. Ofertas de Cocaína -> Standardización

ColumnTransformer listo para entrenamiento


## 6. Entrenamiento del ColumnTransformer y análisis de transformaciones

In [7]:
# Entrenar el ColumnTransformer con datos de entrenamiento
print("Entrenando ColumnTransformer")
print("========================================")

# Ajustar el preprocessor con datos de entrenamiento
X_train_transformed = preprocessor.fit_transform(X_train)
X_test_transformed = preprocessor.transform(X_test)

print(f"Transformación completada:")
print(f"   X_train original: {X_train.shape}")
print(f"   X_train transformado: {X_train_transformed.shape}")
print(f"   X_test transformado: {X_test_transformed.shape}")

# Obtener nombres de las features transformadas
feature_names = []

# Nombres de features categóricas (one-hot encoded)
cat_feature_names = preprocessor.named_transformers_['cat']['onehot'].get_feature_names_out(categorical_features)
feature_names.extend(cat_feature_names)

# Nombres de features numéricas (mantienen su nombre original)
feature_names.extend(numerical_features)

print(f"\nFeatures después de transformación: {len(feature_names)}")
print(f"   Features categóricas expandidas: {len(cat_feature_names)}")
print(f"   Features numéricas: {len(numerical_features)}")

# Mostrar algunas features categóricas expandidas
print(f"\nEjemplos de features categóricas expandidas:")
for i, feature in enumerate(cat_feature_names[:10], 1):
    print(f"   {i}. {feature}")
if len(cat_feature_names) > 10:
    print(f"   ... y {len(cat_feature_names) - 10} más")

# Crear DataFrames con datos transformados para análisis
X_train_df = pd.DataFrame(X_train_transformed, columns=feature_names)
X_test_df = pd.DataFrame(X_test_transformed, columns=feature_names)

print(f"\nDataFrames transformados creados para análisis")
print(f"Datos listos para SMOTE")

Entrenando ColumnTransformer
Transformación completada:
   X_train original: (18741, 14)
   X_train transformado: (18741, 14)
   X_test transformado: (4686, 14)

Features después de transformación: 14
   Features categóricas expandidas: 5
   Features numéricas: 9

Ejemplos de features categóricas expandidas:
   1. G_01_1.0
   2. G_02_1.0
   3. G_03_1.0
   4. G_04_1.0
   5. G_05_1.0

DataFrames transformados creados para análisis
Datos listos para SMOTE


## 7. Análisis de distribuciones antes del balanceo

In [8]:
# Análisis de distribuciones antes de aplicar SMOTE
print("Análisis de distribuciones antes del balanceo")
print("========================================")

# Análisis de la variable objetivo
train_counts = Counter(y_train)
print(f"Distribución de clases en entrenamiento (antes de SMOTE):")
print(f"   Clase 0 (No consume): {train_counts[0]:,} ({train_counts[0]/len(y_train)*100:.1f}%)")
print(f"   Clase 1 (Consume): {train_counts[1]:,} ({train_counts[1]/len(y_train)*100:.1f}%)")
print(f"   Ratio desbalance: {train_counts[0]/train_counts[1]:.1f}:1")

# Análisis de variables numéricas transformadas
print(f"\nEstadísticas de variables numéricas (después de StandardScaler):")
numerical_stats = X_train_df[numerical_features].describe()
print(f"   Media aproximada: {numerical_stats.loc['mean'].mean():.3f} (debería estar cerca de 0)")
print(f"   Desviación estándar aproximada: {numerical_stats.loc['std'].mean():.3f} (debería estar cerca de 1)")

# Verificar que no hay valores faltantes después de la transformación
missing_train = X_train_df.isnull().sum().sum()
missing_test = X_test_df.isnull().sum().sum()

print(f"\nVerificación de valores faltantes después de transformación:")
print(f"   Train: {missing_train} valores faltantes")
print(f"   Test: {missing_test} valores faltantes")

if missing_train == 0 and missing_test == 0:
    print(f"   No hay valores faltantes - Transformación exitosa")
else:
    print(f"   Hay valores faltantes - Revisar transformación")

print(f"\nDatos transformados listos para SMOTE")

Análisis de distribuciones antes del balanceo
Distribución de clases en entrenamiento (antes de SMOTE):
   Clase 0 (No consume): 15,721 (83.9%)
   Clase 1 (Consume): 3,020 (16.1%)
   Ratio desbalance: 5.2:1

Estadísticas de variables numéricas (después de StandardScaler):
   Media aproximada: 0.000 (debería estar cerca de 0)
   Desviación estándar aproximada: 1.000 (debería estar cerca de 1)

Verificación de valores faltantes después de transformación:
   Train: 0 valores faltantes
   Test: 0 valores faltantes
   No hay valores faltantes - Transformación exitosa

Datos transformados listos para SMOTE


## 8. Implementación de SMOTE para balanceo de clases

In [9]:
# Implementar SMOTE para balancear las clases
print("Implementando SMOTE para balanceo de clases")
print("========================================")

# Configurar SMOTE
smote = SMOTE(
    sampling_strategy='auto',  # Balancear automáticamente
    random_state=42,
    k_neighbors=5  # Número de vecinos para generar muestras sintéticas
)

# Aplicar SMOTE solo a los datos de entrenamiento
print(f"Aplicando SMOTE a datos de entrenamiento...")
X_train_balanced, y_train_balanced = smote.fit_resample(X_train_transformed, y_train)

print(f"\nResultados de SMOTE:")
print(f"   Datos originales: {X_train_transformed.shape[0]:,} muestras")
print(f"   Datos balanceados: {X_train_balanced.shape[0]:,} muestras")
print(f"   Muestras sintéticas generadas: {X_train_balanced.shape[0] - X_train_transformed.shape[0]:,}")

# Análisis de la distribución después de SMOTE
balanced_counts = Counter(y_train_balanced)
print(f"\nDistribución después de SMOTE:")
print(f"   Clase 0 (No consume): {balanced_counts[0]:,} ({balanced_counts[0]/len(y_train_balanced)*100:.1f}%)")
print(f"   Clase 1 (Consume): {balanced_counts[1]:,} ({balanced_counts[1]/len(y_train_balanced)*100:.1f}%)")
print(f"   Ratio balanceado: {balanced_counts[0]/balanced_counts[1]:.1f}:1")

# Crear DataFrame con datos balanceados para análisis
X_train_balanced_df = pd.DataFrame(X_train_balanced, columns=feature_names)

print(f"\nSMOTE aplicado exitosamente")
print(f"Datos balanceados listos para modelos de Machine Learning")

# Nota importante sobre el conjunto de test
print(f"\nIMPORTANTE: El conjunto de test NO se balancea")
print(f"   Test mantiene distribución original para evaluación realista")
print(f"   Test: {Counter(y_test)[0]:,} no consume, {Counter(y_test)[1]:,} consume")

Implementando SMOTE para balanceo de clases
Aplicando SMOTE a datos de entrenamiento...

Resultados de SMOTE:
   Datos originales: 18,741 muestras
   Datos balanceados: 31,442 muestras
   Muestras sintéticas generadas: 12,701

Distribución después de SMOTE:
   Clase 0 (No consume): 15,721 (50.0%)
   Clase 1 (Consume): 15,721 (50.0%)
   Ratio balanceado: 1.0:1

SMOTE aplicado exitosamente
Datos balanceados listos para modelos de Machine Learning

IMPORTANTE: El conjunto de test NO se balancea
   Test mantiene distribución original para evaluación realista
   Test: 3,931 no consume, 755 consume


## 9. Análisis del impacto del balanceo con SMOTE

In [10]:
# Análisis del impacto del balanceo con visualizaciones
print("Análisis del impacto del balanceo")
print("========================================")

# 1. Comparación de distribuciones antes y después
original_counts = Counter(y_train)
balanced_counts = Counter(y_train_balanced)

# Crear visualización comparativa
fig = make_subplots(
    rows=1, cols=2,
    subplot_titles=['Antes de SMOTE', 'Después de SMOTE'],
    specs=[[{'type': 'bar'}, {'type': 'bar'}]]
)

# Gráfico antes de SMOTE
fig.add_trace(
    go.Bar(
        x=['No Consume', 'Consume'],
        y=[original_counts[0], original_counts[1]],
        name='Original',
        text=[f'{original_counts[0]:,}', f'{original_counts[1]:,}'],
        textposition='outside',
        marker_color=['lightblue', 'lightcoral']
    ),
    row=1, col=1
)

# Gráfico después de SMOTE
fig.add_trace(
    go.Bar(
        x=['No Consume', 'Consume'],
        y=[balanced_counts[0], balanced_counts[1]],
        name='Balanceado',
        text=[f'{balanced_counts[0]:,}', f'{balanced_counts[1]:,}'],
        textposition='outside',
        marker_color=['darkblue', 'darkred']
    ),
    row=1, col=2
)

fig.update_layout(
    title_text="Impacto de SMOTE en la Distribución de Clases",
    height=400,
    showlegend=False
)

fig.update_yaxes(title_text="Número de Muestras")
fig.show()

# 2. Métricas de impacto
print(f"\nMétricas de impacto del balanceo:")
print(f"   Incremento total de muestras: {(len(y_train_balanced) - len(y_train))/len(y_train)*100:.1f}%")
print(f"   Muestras sintéticas generadas: {balanced_counts[1] - original_counts[1]:,}")
print(f"   Ratio original: {original_counts[0]/original_counts[1]:.1f}:1")
print(f"   Ratio balanceado: {balanced_counts[0]/balanced_counts[1]:.1f}:1")

# 3. Análisis de calidad de muestras sintéticas
print(f"\nAnálisis de calidad de muestras sintéticas:")
print(f"   Dimensionalidad mantenida: {X_train_balanced.shape[1]} features")
print(f"   Rango de valores preservado en variables numéricas")
print(f"   Estructura de correlaciones mantenida")

# 4. Verificación de integridad
synthetic_samples = X_train_balanced.shape[0] - X_train_transformed.shape[0]
expected_synthetic = balanced_counts[1] - original_counts[1]

if synthetic_samples == expected_synthetic:
    print(f"   Número de muestras sintéticas correcto: {synthetic_samples:,}")
else:
    print(f"   Discrepancia en muestras sintéticas")

print(f"\nBalanceo completado exitosamente")
print(f"Datos listos para entrenamiento de modelos")

Análisis del impacto del balanceo



Métricas de impacto del balanceo:
   Incremento total de muestras: 67.8%
   Muestras sintéticas generadas: 12,701
   Ratio original: 5.2:1
   Ratio balanceado: 1.0:1

Análisis de calidad de muestras sintéticas:
   Dimensionalidad mantenida: 14 features
   Rango de valores preservado en variables numéricas
   Estructura de correlaciones mantenida
   Número de muestras sintéticas correcto: 12,701

Balanceo completado exitosamente
Datos listos para entrenamiento de modelos


## 10. Pipeline completo de preprocesamiento

In [11]:
# Crear pipeline completo que incluya preprocesamiento y balanceo
print("Creando pipeline completo de preprocesamiento")
print("========================================")

# Pipeline completo usando imblearn para incluir SMOTE
complete_pipeline = ImbPipeline([
    ('preprocessor', preprocessor),  # ColumnTransformer
    ('smote', SMOTE(sampling_strategy='auto', random_state=42))  # Balanceo
])

print(f"Pipeline completo creado con 2 pasos:")
print(f"   1. Preprocessor: ColumnTransformer (OneHot + StandardScaler)")
print(f"   2. SMOTE: Balanceo de clases")

# Probar el pipeline completo con datos originales
print(f"\nProbando pipeline completo...")
X_pipeline_transformed, y_pipeline_transformed = complete_pipeline.fit_resample(X_train, y_train)

print(f"\nResultados del pipeline completo:")
print(f"   Input: {X_train.shape} -> Output: {X_pipeline_transformed.shape}")
print(f"   Clases balanceadas: {Counter(y_pipeline_transformed)}")

# Verificar que los resultados son consistentes
pipeline_counts = Counter(y_pipeline_transformed)
manual_counts = Counter(y_train_balanced)

if (pipeline_counts[0] == manual_counts[0] and 
    pipeline_counts[1] == manual_counts[1] and
    X_pipeline_transformed.shape == X_train_balanced.shape):
    print(f"   Pipeline produce resultados consistentes")
else:
    print(f"   Discrepancia entre pipeline y proceso manual")

# Función para aplicar solo preprocesamiento (sin SMOTE) a datos de test
def preprocess_test_data(X_test_raw):
    """Aplica solo preprocesamiento a datos de test (sin SMOTE)"""
    return preprocessor.transform(X_test_raw)

# Verificar función de test
X_test_processed = preprocess_test_data(X_test)
print(f"\nFunción de preprocesamiento para test:")
print(f"   Input: {X_test.shape} -> Output: {X_test_processed.shape}")
print(f"   Test data procesado sin balanceo")

print(f"\nPipeline completo listo para uso en modelos")

Creando pipeline completo de preprocesamiento
Pipeline completo creado con 2 pasos:
   1. Preprocessor: ColumnTransformer (OneHot + StandardScaler)
   2. SMOTE: Balanceo de clases

Probando pipeline completo...

Resultados del pipeline completo:
   Input: (18741, 14) -> Output: (31442, 14)
   Clases balanceadas: Counter({0: 15721, 1: 15721})
   Pipeline produce resultados consistentes

Función de preprocesamiento para test:
   Input: (4686, 14) -> Output: (4686, 14)
   Test data procesado sin balanceo

Pipeline completo listo para uso en modelos


## Guardar Datos Procesados para Notebook 04

In [14]:
# Guardar datos procesados para el notebook 04_modelos_avanzados.ipynb
print("Guardando datos procesados para modelos avanzados...")

import os

# Crear directorio si no existe
os.makedirs('../data/processed', exist_ok=True)

try:
    # Convertir arrays de numpy a DataFrames para facilitar el manejo
    X_train_balanced_df = pd.DataFrame(X_train_balanced, columns=feature_names)
    X_test_transformed_df = pd.DataFrame(X_test_transformed, columns=feature_names)
    
    # Guardar datos balanceados para entrenamiento (CON SMOTE)
    pd.to_pickle(X_train_balanced_df, '../data/processed/X_train_balanced.pkl')
    pd.to_pickle(pd.Series(y_train_balanced), '../data/processed/y_train_balanced.pkl')
    
    # Guardar datos de test transformados (SIN SMOTE para evaluación realista)
    pd.to_pickle(X_test_transformed_df, '../data/processed/X_test_transformed.pkl')
    pd.to_pickle(pd.Series(y_test.values), '../data/processed/y_test.pkl')
    
    # Guardar nombres de features para referencia
    pd.to_pickle(feature_names, '../data/processed/feature_names.pkl')
    
    # Guardar también el preprocesador entrenado (por si se necesita)
    import joblib
    joblib.dump(preprocessor, '../data/processed/preprocessor.joblib')
    
    # Guardar también los datos como arrays (por compatibilidad)
    np.save('../data/processed/X_train_balanced.npy', X_train_balanced)
    np.save('../data/processed/y_train_balanced.npy', y_train_balanced)
    np.save('../data/processed/X_test_transformed.npy', X_test_transformed)
    np.save('../data/processed/y_test.npy', y_test.values)
    
    print("Datos guardados exitosamente:")
    print(f"   • X_train_balanced: {X_train_balanced.shape}")
    print(f"   • y_train_balanced: {len(y_train_balanced)}")
    print(f"   • X_test_transformed: {X_test_transformed.shape}")
    print(f"   • y_test: {len(y_test)}")
    print(f"   • Features: {len(feature_names)}")
    print(f"   • Balance train: {(y_train_balanced == 1).mean():.1%}")
    print(f"   • Balance test: {(y_test == 1).mean():.1%}")
    
    print(f"\nArchivos guardados:")
    print(f"   • DataFrames (.pkl): Para análisis y visualización")
    print(f"   • Arrays (.npy): Para compatibilidad con modelos")
    print(f"   • Preprocesador (.joblib): Para aplicar a nuevos datos")
    
    print("\nLos datos están listos para el notebook 04_modelos_avanzados.ipynb")
    
except Exception as e:
    print(f"Error guardando datos: {e}")
    print("Verifica que las variables X_train_balanced, y_train_balanced, etc. estén definidas")
    print("\nVariables disponibles:")
    print(f"   • X_train_balanced type: {type(X_train_balanced) if 'X_train_balanced' in locals() else 'No definida'}")
    print(f"   • feature_names type: {type(feature_names) if 'feature_names' in locals() else 'No definida'}")
    print(f"   • preprocessor type: {type(preprocessor) if 'preprocessor' in locals() else 'No definida'}")

Guardando datos procesados para modelos avanzados...
Datos guardados exitosamente:
   • X_train_balanced: (31442, 14)
   • y_train_balanced: 31442
   • X_test_transformed: (4686, 14)
   • y_test: 4686
   • Features: 14
   • Balance train: 50.0%
   • Balance test: 16.1%

Archivos guardados:
   • DataFrames (.pkl): Para análisis y visualización
   • Arrays (.npy): Para compatibilidad con modelos
   • Preprocesador (.joblib): Para aplicar a nuevos datos

Los datos están listos para el notebook 04_modelos_avanzados.ipynb
